# SALAMI

##Package imports

In [ ]:
#Basic imports
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import json

# Imports to support MIR tasks
import mirdata
import jams
import jams.display
import essentia.standard as ess
import librosa
import librosa.display

#Import general signal processing modules
from scipy.fftpack import fft
from math import floor

#Import audio display 
import IPython.display as ipd

In [ ]:
print(f'matplotlib version ---> {mpl.__version__}')
print(f'librosa version ---> {librosa.__version__}')

Let's check out mirdata's supported datasets

In [ ]:
print(f'Is SALAMI available in supported datasets by mirdata? --> {"salami" in mirdata.list_datasets()}')

In [ ]:
salami = mirdata.initialize("salami")

In [ ]:
salami.data_home = '/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI'

In [ ]:
print(f'Dataset name ---> {salami.name}')
print(f'Bibtex citation ---> {salami.bibtex}')

In [ ]:
validation = salami.validate()
print(f'Validation check up ---> {validation}')

In [ ]:
salami_ids = salami.track_ids  # Return track IDs
salami_data = salami.load_tracks()  # Load Load all tracks in SALAMI

print(type(salami_data[salami_ids[0]])) # Print class
example_track = salami_data[salami_ids[0]]  # Visualize a single track
print(example_track)

In [ ]:
example_track_audio_path = salami_data[salami_ids[0]].audio_path #track's audio
print(f'Path to example track ---> {example_track_audio_path}')

In [ ]:
example_track_audio = salami_data[salami_ids[0]].audio #track's audio
print(f'The actual audio data (array and the sampling rate) of our example track ---> {example_track_audio}')

In [ ]:
#sound reading and displaying
x, fs = example_track_audio
ipd.display(ipd.Audio(data=x, rate=fs))

In [ ]:
#Array preprocessing
# X, fft computation
X = fft(x) 
#fft size
N = X.size
#Array length
M = len(x)

#Setting figure size
plt.figure(figsize=(20, 10))

#Plotting input sound (x-axis in sec)
plt.subplot(2,1,1)
plt.plot(np.arange(N)/float(fs), x, 'b')
plt.plot(x, label = 'sound waveform')
plt.axis([0, N/float(fs), min(x), max(x)])
plt.ylabel('Amplitude')
plt.xlabel('Time (sec)')
plt.title(str(example_track.audio_path))
plt.autoscale(tight=True)

#Plotting magnitude spectra (x-axis in Hz)
plt.subplot(2,1,2)
plt.plot(np.real(20*np.log10(np.abs(X[:int(M/2)+1]))),'r', lw=1.5)
plt.xlabel('Frequency domain (Hz)')
plt.ylabel('Amplitude (dB)')
plt.title('Magnitude spectra')
plt.autoscale(tight=True)

plt.show()

In [ ]:
y, sr = librosa.load(example_track_audio_path)
chroma = librosa.feature.chroma_cqt(y=y, sr=sr)
sim = librosa.segment.recurrence_matrix(chroma)

fig, ax = plt.subplot_mosaic(
    """
    hSS
    hSS
    .vv
    """)

librosa.display.specshow(sim, ax=ax['S'], x_axis='time', y_axis='time', auto_aspect=False)
ax['S'].label_outer()
ax['S'].sharex(ax['v'])
ax['S'].sharey(ax['h'])

librosa.display.waveshow(y, ax=ax['v'])
ax['v'].label_outer()

#librosa.display.waveshow(y=np.transpose(y), ax=ax['h'])
#ax['h'].label_outer()

plt.autoscale(tight=False)
plt.show()

#lib_dpl.waveshow(y, ax=ax['h'], transpose=True)
#ax['h'].label_outer()

The SALAMI dataset provides two levels of reference structural annotations for each track: an upper-level segmentation, where the structural patterns corresponds to sections, and a lower-level segmentation that can reach the phrase level.

In [ ]:
example_ann_lw1 = example_track.sections_annotator1_lowercase_path
example_ann_up1 = example_track.sections_annotator1_uppercase_path
example_ann_lw2 = example_track.sections_annotator2_lowercase_path
example_ann_up2 = example_track.sections_annotator2_uppercase_path

print(f'Path to sections_annotator1_lowercase_path for sample track --> {example_ann_lw1}')
print(f'Path to sections_annotator1_uppercase_path for sample track --> {example_ann_up1}')
print(f'Path to sections_annotator1_lowercase_path for sample track --> {example_ann_lw2}')
print(f'Path to sections_annotator1_uppercase_path for sample track --> {example_ann_up2}')

In [ ]:
with open(example_ann_lw1) as f:
    contents_lw1 = f.readlines()

with open(example_ann_up1) as f:
    contents_up1 = f.readlines()

with open(example_ann_lw2) as f:
    contents_lw2 = f.readlines()

with open(example_ann_up2) as f:
    contents_up2 = f.readlines()
    
print(f'Actual content for "Annotator 1 lowercase for sample track" --> {contents_lw1}')
print(f'Actual content for "Annotator 1 uppercase for sample track "--> {contents_up1}')   
print(f'Actual content for "Annotator 2 lowercase for sample track" --> {contents_lw2}')
print(f'Actual content for "Annotator 2 lowercase for sample track" --> {contents_up2}')

In [ ]:
example_track.to_jams()

In [ ]:
sorted_key_list = example_track.to_jams().annotations[1].data
print(type(sorted_key_list))
print(sorted_key_list)

In [ ]:
df = pd.DataFrame(sorted_key_list)
df

##Metadata descriptions

`metadata.csv`

This is the main metadata file and describes all the pieces in the collection.

The fields are:

	SONG_ID                   Unique identifier for piece of music
	SOURCE                    Either Codaich, IA (Internet Archive), or RWC
	ANNOTATOR1                ID number for first annotator
	ANNOTATOR2                ID number for second annotator
	SONG_DURATION             Duration of the piece, in seconds
	SONG_TITLE                Title
	ARTIST                    Artist
	ANNOTATION_TIME1          Self-reported time to complete annotation for first annotator
	ANNOTATION_TIME2          Self-reported time to complete annotation for second annotator
	TEXTFILE1                 File path for first annotator's file
	TEXTFILE2                 File path for second annotator's file
	CLASS                     Broad genre (classical, jazz, popular, world, Live_Music_Archive, or unknown)
	GENRE                     Narrow genre
	SUBMISSION_DATE1          Date of submission of first annotation
	SUBMISSION_DATE2          Date of submission of second annotation
	XEQS1                     Was the first annotation converted automatically from X/= notation? X indicates yes; 0 indicates no.
	XEQS2                     Was the second annotation converted automatically from X/= notation? X indicates yes; 0 indicates no.

In [ ]:
metadata_csv = '/content/drive/MyDrive/SALAMI/salami-data-public-hierarchy-corrections/metadata/metadata.csv'
df = pd.read_csv(metadata_csv)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
for index, track in enumerate(salami_data):
  print(f'Iteration --> {index}', 
        f'Track data --> {salami_data[salami_ids[int(index)]]}', 
        f'JAMS annotaion --> {salami_data[salami_ids[int(index)]].to_jams()}')